In [ ]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
from datetime import datetime as datetime
import matplotlib.pyplot as plt

client = MongoClient("mongodb://localhost:27017")

db = client["lifesnaps"] 
collection = db["fitbit"]

userID = ObjectId("621e2f9167b776a240011ccb")




In [ ]:
qry = {
    "id": ObjectId("621e2f9167b776a240011ccb"),
    "type": "resting_heart_rate",
    }
  
projection = {
    "id":1, 
    "data.dateTime": 1, 
    "data.value.value": 1, 
    "data.value.error":1, 
    "_id":0
    }

rawDocs = list(collection.find(qry,projection))

In [ ]:
# Convert to DataFrame
flatDocs = []
for doc in rawDocs:
    data = doc.get("data",{})
    value = data.get("value",{})
    
 # confidence van 0 niet inbegrepen -> te onzeker
    flatDocs.append({
        "dateTime": data["dateTime"],
        "restingHeartRate": value["value"],
        "errorRHR": value["error"],
    })


df_RHR = pd.DataFrame(flatDocs)
df_RHR["dateTime"] = pd.to_datetime(df_RHR["dateTime"], errors='coerce')
df_RHR["restingHeartRate"] = pd.to_numeric(df_RHR["restingHeartRate"], errors='coerce')
df_RHR["errorRHR"] = pd.to_numeric(df_RHR["errorRHR"], errors='coerce')
df_RHR = df_RHR.sort_values(by='dateTime', ascending=True)
df_RHR.head(70)



In [ ]:
(df_RHR == 0).describe()

In [ ]:
(df_RHR == 0).sum() # 4x 0-waarden

upper = df_RHR["restingHeartRate"] + df_RHR["errorRHR"]
lower = df_RHR["restingHeartRate"] - df_RHR["errorRHR"]

plt.plot(df_RHR["dateTime"], df_RHR["restingHeartRate"], label="restingHeartRate")
plt.fill_between(df_RHR["dateTime"], lower, upper, color='tab:purple', alpha=0.2, label="± error")
plt.legend()

In [ ]:
df_RHR.rename(columns={'dateTime': 'date'}, inplace=True)
df_RHR.to_csv('csv/resting_heart_rate.csv',index=False)

df_RHR
